In [61]:
import os
import torch
import pandas as pd
import easyocr
from PIL import Image
import numpy as np
from torchmetrics.functional import char_error_rate, word_error_rate
import re

In [4]:
def process_image(reader, img_path: str):
    image = Image.open(img_path)
    image = np.array(image)
    bounds = reader.readtext(image, paragraph=True)
    answer = postprocess(bounds)
    return answer

def postprocess(bounds):
    main_ingredient_list = ""
    for bound in bounds:
        main_ingredient_list += bound[1].lower() + ' '

    # Remove unnecessary punctuation and other symbols
    unnecessary_symbols = ";:.,!?%#\"'/\\"
    for sym in unnecessary_symbols:
        main_ingredient_list = main_ingredient_list.replace(sym, '')

    # Remove extra spaces
    main_ingredient_list = " ".join(main_ingredient_list.split(' '))

    return main_ingredient_list

In [5]:
reader_old = easyocr.Reader(['ru'])
reader_new = easyocr.Reader(['ru'],
                        model_storage_directory='custom_EasyOCR/model',
                        user_network_directory='custom_EasyOCR/user_network',
                        recog_network='custom_example') 

In [6]:
df = pd.read_csv('dataset/test/labels.csv')

In [7]:
df

,filename,words
0,image_115.png,лрите на бутылке . хранить в сухом; тем
1,image_1473.png,олее 30 мг на 100 мл напитка. не рокомендется ...
2,image_1059.png,"ул. дзержинского, 19"
3,image_803.png,и относительной вложности возд
4,image_390.png,"белки 8,3 г; жиры 16,5"
...,...,...
220,image_781.png,"сливочны \r\n991,image_782.png,не более 75%. п..."
221,image_347.png,мл
222,image_257.png,бензоат
223,image_915.png,нный категории 6 охлажденный упа


In [8]:
list_images = df['filename'].to_list()

In [45]:
list_words = df['words'].to_list()

In [62]:
preds_old = []
preds_new = []
target = []

In [63]:
for i in range(len(list_images)):
    img_path='dataset/test/'+list_images[i]
    text_old = process_image(reader_old, img_path)
    text_new = process_image(reader_new, img_path)  
    text_gr_truth = re.sub(r"[!@#&*()[]{}/?<>;:.,#\"/\\]", "", list_words[i])
    preds_old.append(text_old)
    preds_new.append(text_new)
    target.append(text_gr_truth)

In [69]:
char_error_rate_old = char_error_rate(preds=preds_old, target=target)
char_error_rate_new = char_error_rate(preds=preds_new, target=target)
print(char_error_rate_old, char_error_rate_new)

tensor(0.1954) tensor(0.1633)
